## Self driving car model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import cv2

In [ ]:
### Dataset
use_original = False
import os

def get_dataset(use_original=True):
    if use_original:
        csv_filepath = './data/driving_log.csv'
        img_filepath = './data/IMG/'
    else:
        csv_filepath = './data_custom/driving_log.csv'
        img_filepath = './data_custom/IMG/'

    df = pd.read_csv(csv_filepath, header=None)
    lines = df.values.tolist()

    images_center = []
    measurements_center = []
    images_lr = []
    measurements_lr = []

    for line in lines:
        source_path = line[0]
        filename = os.path.basename(source_path)
        current_path = os.path.join(img_filepath, filename)
        image = cv2.imread(current_path)
        images_center.append(image)
        measurement = float(line[3])
        measurements_center.append(measurement)

        for source_path in line[1:3]:
            filename = os.path.basename(source_path)
            current_path = os.path.join(img_filepath, filename)
            measurement = float(line[3])

            if measurement != 0:
                if source_path == line[1]:
                    measurement += 0.23
                else:
                    measurement -= 0.23

                image = cv2.imread(current_path)
                images_lr.append(image)
                measurements_lr.append(measurement)

    measurements_center = np.asarray(measurements_center)
    images_center = np.asarray(images_center)
    measurements_lr = np.asarray(measurements_lr)
    images_lr = np.asarray(images_lr)

    images = np.concatenate([images_center, images_lr], axis=0)
    measurements = np.concatenate([measurements_center, measurements_lr], axis=0)

    if use_original:
        print('Udacity data is loaded')
    else:
        print('Custom data is loaded')

    print('Dataset size: %d' % len(measurements))

    return images, measurements

